### Load Input Data

In [1]:
PROCESSED_DATA = "./processed-data"

import torch
import pandas as pd
import numpy as np

dataset = pd.read_csv('{}/latestSequence.csv'.format(PROCESSED_DATA), header = 0)
dataset.set_index(dataset.columns[0], inplace=True)
print(dataset[-24:])
input_data = np.array(dataset)
mean = np.mean(input_data, axis=0)
std = np.std(input_data, axis=0) 
input_data = (input_data - mean)/std  
input_data = torch.Tensor(input_data).unsqueeze(0)
print(input_data.shape)

                  Temp (°C)  Dew Point Temp (°C)  Rel Hum (%)  \
Date/Time                                                       
2019-08-08 00:00       21.2                 21.2        100.0   
2019-08-08 01:00       21.3                 21.2        100.0   
2019-08-08 02:00       21.6                 21.6        100.0   
2019-08-08 03:00       20.7                 20.7        100.0   
2019-08-08 04:00       18.8                 18.8        100.0   
2019-08-08 05:00       18.3                 17.8         97.0   
2019-08-08 06:00       18.6                 18.0         96.0   
2019-08-08 07:00       20.5                 17.1         81.0   
2019-08-08 08:00       21.7                 15.2         66.0   
2019-08-08 09:00       22.9                 16.7         68.0   
2019-08-08 10:00       23.3                 14.1         56.0   
2019-08-08 11:00       25.2                 22.5         85.0   
2019-08-08 12:00       24.4                 20.0         76.0   
2019-08-08 13:00       23

### Set Model

In [2]:
import torch
import torch.nn as nn

class SimpleLSTM(nn.Module):
    def __init__(self, input_size = 8, output_size = 1, hidden_size=100, num_layers=1):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True) 
        self.fc = nn.Linear(self.hidden_size, self.output_size) 

    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, cell
    
    def forward(self, x):
        # hidden, cell state init
        h, c = self.init_hidden(x.size(0))
        h, c = h.to(x.device), c.to(x.device)
        out, (h, c) = self.lstm(x, (h, c))     
        final_output = self.fc(out[:, -1:, :])     
        final_output = torch.squeeze(final_output, dim = 1) # shape (100,1)

        return final_output

### Load Model and Predict  

The input data are 24 set of 480 hours data till 00, 01, ...,  23 hour on Aug. 8th   
The predictions are temperatures at 00 hour, 01 hour, ..., 23 hour on Aug. 9th

In [3]:
model_path = './saved/LSTM/best_model.pt'
model = SimpleLSTM()

checkpoint = torch.load(model_path)    
state_dict = checkpoint['net']   
model.load_state_dict(state_dict) 


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [4]:
preds = []

for count in range(0,24):
    pred = model(input_data[:, count:480+count, :])
    pred = pred.item() * std[0] + mean[0] # de-normalization
    preds.append(pred)

count = 0
for pred in preds:
    print("{:02d} - {:2.2f}".format(count, pred))
    count = count+1

# The actual temp was 17.0
# http://climate.weather.gc.ca/climate_data/hourly_data_e.html?hlyRange=2008-07-15%7C2019-08-07&dlyRange=2008-07-15%7C2019-08-07&mlyRange=%7C&StationID=47267&Prov=ON&urlExtension=_e.html&searchType=stnProv&optLimit=specDate&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=74&Month=8&Day=1&lstProvince=ON&timeframe=1&Year=2019


00 - 21.67
01 - 21.59
02 - 21.58
03 - 21.59
04 - 21.55
05 - 21.45
06 - 21.35
07 - 21.35
08 - 21.36
09 - 21.45
10 - 21.51
11 - 21.61
12 - 21.68
13 - 21.71
14 - 21.53
15 - 21.45
16 - 21.36
17 - 21.36
18 - 21.41
19 - 21.44
20 - 21.36
21 - 21.19
22 - 20.94
23 - 20.62


### Summary

1) LSTM Model was trained on Kingston Climate Station's data between 2015.1.1. ~ 2019.8.7  
   Weakness - The prediction is dependent on only one station in Kingston  
   
2) Model gets 480 hours data as input and predicts the temperature 24 hours later  

3) Prediction for Aug 9th doesn't properly describe temp decreases at night and early morning even though hour is added as feature